In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import os
from matplotlib import pyplot as plt
%matplotlib inline

/home/atul/keras_opencv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

# Import Data and Preprocessing

In [3]:
def load_data(target):
    names=os.listdir(target)
    x=[]
    y=[]
    for i in names:
        path_target=os.path.join(target,i)
        path_image=os.listdir(path_target)
        for j in path_image:
            path_final=os.path.join(path_target,j)
            img=cv2.imread(path_final,0)
            img=cv2.resize(img,(32,32))
            x.append(img)
            y.append(i)
    return x,y

In [4]:
x_train,y_train=load_data("train/")
x_test,y_test=load_data("validation/")

In [5]:
x_train=np.array(x_train).astype("float32")
y_train=np.array(y_train)
x_test=np.array(x_test).astype("float32")
y_test=np.array(y_test)

In [6]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(4844, 32, 32) (4844,) (1619, 32, 32) (1619,)


In [7]:
plt.figure(figsize=[200,200])
for i in range(1,4800,100):
    plt.subplot(10,5,i/100 +1)
    plt.imshow(x_train[i])
    plt.title(y_train[i])
    plt.xticks([]), plt.yticks([]) 

In [8]:
def sub(data):
    data=pd.DataFrame(data,columns=["type"])
    a={'Apple Braeburn':0, 'Apple Golden 1':1, 'Apple Golden 2':2,
       'Apple Golden 3':3, 'Apple Granny Smith':4, 'Apple Red 1':5,
       'Apple Red 2':6, 'Apple Red 3':7, 'Apple Red Delicious':8,
       'Apple Red Yellow':9}
    data= data["type"].map(a)
    return data.values
    

In [9]:
y_train=sub(y_train)
y_test=sub(y_test)

In [10]:
x_train=x_train/255
y_train=y_train/255

In [11]:
y_train=y_train.astype("int32")
y_test=y_test.astype("int32")

# CNN Function

In [19]:

def cnn_model_fn(features, labels, mode):
    
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.layers.flatten(pool2)
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
 #   dense2 = tf.layers.dense(inputs=dropout1, units=512, activation=tf.nn.relu) 
 #   dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout1, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



# Create Customized Estimator, train and Evaluate

In [21]:
apple_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/apple_class2")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94ed08be80>, '_tf_random_seed': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_model_dir': '/tmp/apple_class2', '_save_checkpoints_secs': 600, '_is_chief': True, '_service': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000}


In [22]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": x_train},
      y=y_train,
      batch_size=100,
      num_epochs=None,shuffle=True)

In [ ]:
apple_classifier.train(input_fn=train_input_fn, steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/apple_class2/model.ckpt.
INFO:tensorflow:loss = 2.29058, step = 1
INFO:tensorflow:global_step/sec: 1.86337
INFO:tensorflow:loss = 0.049360655, step = 101 (53.667 sec)
INFO:tensorflow:global_step/sec: 1.91773
INFO:tensorflow:loss = 0.010463454, step = 201 (52.145 sec)
INFO:tensorflow:global_step/sec: 2.11593
INFO:tensorflow:loss = 0.0042656153, step = 301 (47.261 sec)
INFO:tensorflow:global_step/sec: 2.13702
INFO:tensorflow:loss = 0.002764766, step = 401 (46.794 sec)
INFO:tensorflow:global_step/sec: 2.15244
INFO:tensorflow:loss = 0.0024860322, step = 501 (46.459 sec)
INFO:tensorflow:global_step/sec: 2.17375
INFO:tensorflow:loss = 0.0014204744, step = 601 (46.003 sec)
INFO:tensorflow:global_step/sec: 2.1567
INFO:tensorflow:loss = 0.0010395627, step = 701 (46.367 sec)
INFO:tensorflow:global_step/sec: 2.12865
INFO:tensorflow:loss = 0.0007555303, step = 801 (46.978 sec)
INFO:tensorflow:global_step

INFO:tensorflow:loss = 0.00014456111, step = 7501 (38.440 sec)
INFO:tensorflow:global_step/sec: 2.59132
INFO:tensorflow:loss = 8.4194595e-05, step = 7601 (38.590 sec)
INFO:tensorflow:global_step/sec: 2.59213
INFO:tensorflow:loss = 0.00015914341, step = 7701 (38.578 sec)
INFO:tensorflow:global_step/sec: 2.60273
INFO:tensorflow:loss = 0.00017221479, step = 7801 (38.421 sec)
INFO:tensorflow:global_step/sec: 2.60744
INFO:tensorflow:loss = 0.00015022054, step = 7901 (38.352 sec)
INFO:tensorflow:global_step/sec: 2.59892
INFO:tensorflow:loss = 9.4911506e-05, step = 8001 (38.478 sec)
INFO:tensorflow:global_step/sec: 2.60568
INFO:tensorflow:loss = 9.1135225e-05, step = 8101 (38.377 sec)
INFO:tensorflow:global_step/sec: 2.5999
INFO:tensorflow:loss = 6.49943e-05, step = 8201 (38.463 sec)
INFO:tensorflow:global_step/sec: 2.50697
INFO:tensorflow:loss = 5.2996344e-05, step = 8301 (39.889 sec)
INFO:tensorflow:global_step/sec: 1.83852
INFO:tensorflow:loss = 6.727471e-05, step = 8401 (54.392 sec)
INFO:

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
eval_results = apple_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


In [ ]:
y_test.shape

In [ ]:
x_test.shape